In [1]:
from casadi import *


In [2]:
# T for total steps
T = 15
N_rope = 4

In [3]:

# force applied to vec1
# ie force that moves vec1 towards vec2
# Fs=-k*(mag(L)-L0)*norm(L)

vec = MX.sym('vec1',2)

# vector length

v_len = Function('f',[vec],\
           [ sqrt(vec[0]**2 + vec[1]**2 + 0.000001)   ])

# add 0.00001 so you don't divide by 0

v_norm = Function('f',[vec],\
           [ reshape((vec / (v_len(vec) + 0.000001)),1,2)  ])

print(v_len(DM([3,4])))
print(v_norm(DM([3,4])))


5
[[0.6, 0.8]]


In [4]:
# Fs=-k*(mag(L)-L0)*norm(L)
spring_k=0.1
l=10
F_spring = Function('f',[vec],\
           [ -spring_k*( v_len(vec)  - l) * v_norm(vec) ])


In [5]:
# test = DM([[1,2],[4,5],[1,2],[4,5]])
test = DM([[0,0],[0,0],[0,0],[0,0]])
print(test)


for i in range(0,(4) * 2):
    test[i] += i

# test += vertcat(DM([1,1,1]),DM.zeros(3,2))
print(test)


[[0, 0], 
 [0, 0], 
 [0, 0], 
 [0, 0]]

[[0, 4], 
 [1, 5], 
 [2, 6], 
 [3, 7]]


In [6]:


opti = casadi.Opti()
# States
# N_rope = 10
# d_rope = 10

# for each time step (plus an initial one), for each segment, there is 2 numbers (x and y) 
pos = opti.variable(T+1,N_rope * 2)

# input
u = opti.variable(1,2)


In [7]:
print(list(range(N_rope)))
print(DM.zeros(N_rope*2))
print(pos[0,:])



opti.subject_to(pos[0,:]==DM.zeros(1,N_rope * 2))

[0, 1, 2, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
opti0_x_1[:120:15]


In [8]:
def one_hot(i):
    global N_rope
    return DM([(1 if a == i else 0) for a in range(2 * N_rope) ]).T
print(one_hot(9 + N_rope))

[[0, 0, 0, 0, 0, 0, 0, 0]]


In [9]:
# k_pos = pos[1,:]
# print(k_pos.shape)
# print(k_pos[N_rope].shape)

# print(u.shape)
# print(u[1])

for k in range(1,T):
    prev_pos = pos[k-1,:]


    d_pos = DM.zeros(1,N_rope * 2)
    if k != 1:
#         verlet integration on all rope segments
         d_pos += pos[k-1,:]-pos[k-2,:]
    
######     Constraints

#   k_pos[0] is the x coord of the first rope segment
#   k_pos[0 + N_rope] accesses the y coord
    
    d_pos += one_hot(0)          * (u[0] - prev_pos[0])          * 0.25
    d_pos += one_hot(0 + N_rope) * (u[1] - prev_pos[0 + N_rope]) * 0.25
    
    for i in range(1,N_rope - 1):
        v1 = (prev_pos[i]  -  prev_pos[i-1]) * DM([1,0]) + (prev_pos[i+N_rope] - prev_pos[(i-1) + N_rope]) * DM([0,1])
        v2 = (prev_pos[i]  -  prev_pos[i+1]) * DM([1,0]) + (prev_pos[i+N_rope] - prev_pos[(i+1) + N_rope]) * DM([0,1])
        F = F_spring(v1) + F_spring(v2)
        d_pos += one_hot(i)          * F[0]
        d_pos += one_hot(i + N_rope) * F[1]
    
    v1 = (prev_pos[(N_rope - 1)]  -  prev_pos[(N_rope - 1)-1]) * DM([1,0]) + (prev_pos[(N_rope - 1)+N_rope] - prev_pos[((N_rope - 1)-1) + N_rope]) * DM([0,1])
    F = F_spring(v1)
    d_pos += one_hot(N_rope - 1)          * F[0]
    d_pos += one_hot(N_rope - 1 + N_rope) * F[1]
    
    opti.subject_to(pos[k,:]==prev_pos + d_pos)


In [10]:
final_pos = pos[T,:]

goal = 0
for i in range(T):
    i_x = pos[i,:][(N_rope - 1)]
    i_y = pos[i,:][(N_rope - 1) + N_rope]
    goal += ((i_x - 50)**2 + (i_y-60)**2 ) * (i/T)

# final_x = final_pos[(N_rope - 1)]
# final_y = final_pos[(N_rope - 1) + N_rope]
# goal = (final_x - 50)**2 + (final_y-60)**2 
opti.minimize(goal)

opti.subject_to(opti.bounded(0,u[0],100));
opti.subject_to(opti.bounded(0,u[1],100));

opti.set_initial(u[0], 75)
opti.set_initial(u[1], 75)



opti.solver('ipopt');
sol = opti.solve();

RuntimeError: .../casadi/core/sparsity_internal.cpp:2150: Assertion "in_range(rr, -size1()+ind1, size1()+ind1)" failed:
Out of bounds error. Got elements in range [15,15], which is outside the range [-15,15).

In [ ]:
opti.debug

In [ ]:
print(final_spiderman_x)